## Grid Search framewrok for sarima hyperparameters for sales prediction

Import Libraries

In [ ]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_excel

one-step sarima forecast

In [ ]:
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

root mean squared error or rmse

In [ ]:
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

split a univariate dataset into train/test sets

In [ ]:
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

walk-forward validation for univariate data

In [ ]:
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

score a model, return None on failure

In [ ]:
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)


grid search configs

In [ ]:
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

create a set of sarima configs to try

In [ ]:
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models


In [ ]:
if __name__ == '__main__':
	# load dataset
	series = read_excel('/content/Sales-monthly.xlsx', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = sarima_configs(seasonal=[0,6,12])
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[[(0, 0, 0), (0, 0, 0, 6), 'n']] 171790.149
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 171790.149
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 171790.149
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 196516.436
 > Model[[(0, 0, 0), (0, 0, 1, 6), 'n']] 171665.990
 > Model[[(0, 0, 0), (0, 1, 0, 6), 'n']] 170347.702
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 64073.694
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 192416.122
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 179742.005
 > Model[[(0, 0, 0), (1, 0, 0, 6), 'n']] 202149.876
 > Model[[(0, 0, 0), (0, 1, 1, 6), 'n']] 99783.312
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 107991.687
 > Model[[(0, 0, 0), (1, 0, 1, 6), 'n']] 173695.376
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 185536.266
 > Model[[(0, 0, 0), (1, 1, 0, 6), 'n']] 67805.053
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'n']] 167404.944
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 179531.238
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 180513.770
 > Model[[(0, 0, 0), (1, 1, 1, 6), 'n']] 144234.775
 > Model[[(